In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys; sys.path.append("../")

import time
import lzma
import pickle
import pickle
from tqdm import tqdm
from SALib.sample import saltelli
from network_model_cpt_intervention import *


import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15,
                     "figure.dpi" : 100, 
                     "grid.alpha" : 0.3, 
                     "axes.grid": True, 
                     "axes.axisbelow" : True,
                     "figure.figsize":(8,6),
                     "mathtext.fontset":"cm",
                     "xtick.labelsize": 14,
                     "ytick.labelsize": 14,
                     "axes.labelsize": 16, 
                     "legend.fontsize": 13.5})
USE_TEX = False
if USE_TEX:
    plt.rc("text", usetex=True)
    plt.rc("text.latex", preamble=r"""
     \usepackage{times}
     \usepackage{mathptmx}""")
else:
    plt.rc("text", usetex=False)
plt.rc("font", family="serif")

In [ ]:
# load one of the random seed
SEED = 1
data = pickle.load(lzma.open(f"../data/wealth_arrays/concat_W_arrays_cpt/{SEED}_8192_1250_51_cpt.pkl.lzma"))


# pick a model run from category 5 of CPT / HK that somewhat reaches equilibrium
RUN = 127
W = data[RUN].T
plt.plot(W)
plt.show()

In [ ]:
mean_deltas = []
for t in range(1,51):
    delta = np.abs((W[t] - W[t-1]) / W[t-1])
    delta = np.where(np.isnan(delta), 0, delta)
    mean_deltas.append(np.mean(delta))
    
    
window = 5
smoothed_deltas = [np.mean(mean_deltas[i:i+window]) for i in range(len(mean_deltas)-window)]
plt.plot(smoothed_deltas)
plt.show()

In [ ]:
# identify bottom 7%
bottom_7_percent = np.argsort(W[-1])[:int(0.07*len(W[-1]))]
bottom_7_percent

In [ ]:
# determine budget B
B = sum(1.0 - W[-1][bottom_7_percent])
B

# Category 5 run

In [ ]:
# load graph, and extract communities, community_membership
with open("../graph.pickle", "rb") as f:
    graph = pickle.load(f)
communities = get_communities(graph)
community_membership = get_community_membership(graph, communities)


# problem definition
PROBLEM = {
    "num_vars" : 6,
    "names"    : ["project_cost",
                  "gain_right",
                  "alpha_beta",
                  "prob_left",
                  "init_w_scale",
                  "poisson_scale"],
    "bounds"   : [[0.01, 2.00],
                  [1.70, 2.30],
                  [0.70, 0.80],
                  [0.30, 0.45],
                  [0.01, 0.15],
                  [8.00, 20.0]]
}

# generate Saltelli samples
NUM_SAMPLES = 1024
X = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)
row = X[127]

# run simulation
W, A, U, P, T, _, G, bottom_7_percent = simulation (
    graph=graph,
    communities=communities,
    community_membership=community_membership,
    NUM_AGENTS=1250,
    STEPS=100,
    seed=2,
    PROJECT_COST=row[0],
    gain_right=row[1],
    ALPHA_BETA=row[2],
    prob_left=row[3],
    init_wealth_scale=row[4],
    poisson_scale=row[5]
)

In [ ]:
# STIMULUS = 1.0
plt.plot(W[:,bottom_7_percent])
plt.show()

In [ ]:
# STIMULUS = 3.0
plt.plot(W[:,bottom_7_percent])
plt.show()

In [ ]:
# STIMULUS = 5.0
plt.plot(W[:,bottom_7_percent])
plt.show()

In [ ]:
# STIMULUS = 7.0
plt.plot(W[:,bottom_7_percent])
plt.show()

# Category 1 run

In [ ]:
# load graph, and extract communities, community_membership
with open("../graph.pickle", "rb") as f:
    graph = pickle.load(f)
communities = get_communities(graph)
community_membership = get_community_membership(graph, communities)


# problem definition
PROBLEM = {
    "num_vars" : 6,
    "names"    : ["project_cost",
                  "gain_right",
                  "alpha_beta",
                  "prob_left",
                  "init_w_scale",
                  "poisson_scale"],
    "bounds"   : [[0.01, 2.00],
                  [1.70, 2.30],
                  [0.70, 0.80],
                  [0.30, 0.45],
                  [0.01, 0.15],
                  [8.00, 20.0]]
}

# generate Saltelli samples
NUM_SAMPLES = 1024
X = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)
row = X[719]


results = []
for STIMULUS in [1.0, 3.0, 5.0, 7.0]:
    # run simulation
    W, A, U, P, T, _, G, bottom_7_percent = simulation (
        communities=communities,
        community_membership=community_membership,
        NUM_AGENTS=1250,
        STEPS=100,
        seed=2,
        PROJECT_COST=row[0],
        gain_right=row[1],
        ALPHA_BETA=row[2],
        prob_left=row[3],
        init_wealth_scale=row[4],
        poisson_scale=row[5], 
        stimulus=STIMULUS
    )
    results.append((W))

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(12,4))
titles = [f"Simulus = {s}" for s in [1.0, 3.0, 5.0, 7.0]]

for i,r in enumerate(results):
    axs[i].plot(W[:,bottom_7_percent])
    axs[i].set_title(titles[i])